In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
# Select Product Code 841810, 841840, or 841850
Product=841810

In [2]:
#'My number is: {one}, and my name is: {two}'.format(one=num,two=name)
'UK Import Mirror Data - Unit Value\{}-2015-UK-Import-mirror-Unit-Value.txt'.format(Product)

'UK Import Mirror Data - Unit Value\\841840-2015-UK-Import-mirror-Unit-Value.txt'

In [4]:
#Butun Dosyalar okunur Dosya isimleri duzenlenmelidir.
UK_16 = pd.read_csv('UK Import Mirror Data - Unit Value\{}-2016-UK-Import-mirror-Unit-Value.txt'.format(Product),sep='\t')
UK_15 = pd.read_csv('UK Import Mirror Data - Unit Value\{}-2015-UK-Import-mirror-Unit-Value.txt'.format(Product),sep='\t')
UK_13 = pd.read_csv('UK Import Mirror Data - Unit Value\{}-2013-UK-Import-mirror-Unit-Value.txt'.format(Product),sep='\t')
UK_12 = pd.read_csv('UK Import Mirror Data - Unit Value\{}-2012-UK-Import-mirror-Unit-Value.txt'.format(Product),sep='\t')
UK_10 = pd.read_csv('UK Import Mirror Data - Unit Value\{}-2010-UK-Import-mirror-Unit-Value.txt'.format(Product),sep='\t')
UK_09 = pd.read_csv('UK Import Mirror Data - Unit Value\{}-2009-UK-Import-mirror-Unit-Value.txt'.format(Product),sep='\t')
UK_07 = pd.read_csv('UK Import Mirror Data - Unit Value\{}-2007-UK-Import-mirror-Unit-Value.txt'.format(Product),sep='\t')
UK_05 = pd.read_csv('UK Import Mirror Data - Unit Value\{}-2005-UK-Import-mirror-Unit-Value.txt'.format(Product),sep='\t')
UK_04 = pd.read_csv('UK Import Mirror Data - Unit Value\{}-2004-UK-Import-mirror-Unit-Value.txt'.format(Product),sep='\t')

In [90]:
#Butun Importer bilgisine gore birlestirilir.
UK_all = UK_16.merge(UK_15, 
                       
                        on = 'Exporters',how = 'left').merge(UK_13,
                        on = 'Exporters',how = 'left').merge(UK_12, 
                        on = 'Exporters',how = 'left').merge(UK_10,
                        on = 'Exporters',how = 'left').merge(UK_09, 
                        on = 'Exporters',how = 'left').merge(UK_07,
                        on = 'Exporters',how = 'left').merge(UK_05,
                        on = 'Exporters',how = 'left').merge(UK_04,
                        on = 'Exporters',how = 'left')


In [91]:
#O urun grubundaki en fazla hacim yapan ülkelere bakılır. 
#2016 yılı sonu itibariyle en cok ihracat yapılan ulkeler secilmistir
UK_all['Exporters'].unique()

array(['Turkey', 'Korea, Republic of', 'United States of America',
       'Lithuania', 'Mexico', 'Netherlands', 'Norway', 'Philippines',
       'Poland', 'Portugal', 'Romania', 'Serbia', 'India', 'Singapore',
       'Slovakia', 'Slovenia', 'South Africa', 'Spain', 'Switzerland',
       'Thailand', 'Australia', 'Austria', 'Belgium', 'Bermuda',
       'Bulgaria', 'Canada', 'China', 'Czech Republic', 'Denmark',
       'Estonia', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland',
       'Italy'], dtype=object)

In [92]:
#Butun okunanların birlestirir, transposunu alır ve index'i kaldırır Bunu urun grubuna göre yapar
# 841810 için ilk 5 ülke
#'World', 'China', 'Poland', 'Turkey', 'Korea, Republic of', 'Italy'
if Product==841810 :
    UK_all=UK_all[(
    
    (UK_all['Exporters']=='Poland') 
   | (UK_all['Exporters']=='Korea, Republic of')
   | (UK_all['Exporters']=='Italy')
   | (UK_all['Exporters']=='Turkey')
   | (UK_all['Exporters']=='China')        
    )]
    columnnames=UK_all.columns[UK_all.columns.str.contains("Unit")==True]
    columnnames=columnnames.append(UK_all.columns[UK_all.columns.str.contains("Imported value")==True])
    columnnames=columnnames.append(UK_all.columns[UK_all.columns.str.contains("Imported quantity")==True])
    
  
    UK_all.drop(columnnames, axis=1,inplace=True)
    UK_all=UK_all.transpose().reset_index()
# 841840 için ilk 5 ülke
#''World', 'China', 'Turkey', 'Germany', 'Netherlands', 'Hungary',
elif Product==841840:
    UK_all=UK_all[(
    (UK_all['Exporters']=='World')  
   | (UK_all['Exporters']=='Germany') 
   | (UK_all['Exporters']=='Netherlands')
   | (UK_all['Exporters']=='Hungary')
   | (UK_all['Exporters']=='Turkey')
   | (UK_all['Exporters']=='China')        

    )]
    columnnames=UK_all.columns[UK_all.columns.str.contains("Unit")==True]
    UK_all.drop(columnnames, axis=1,inplace=True)
    UK_all=UK_all.transpose().reset_index()


# 841850 için ilk 5 ülke
#'World', 'China', 'Turkey', 'Austria', 'Czech Republic', 'Italy',
#       

elif Product==841850:
    UK_all=UK_all[(
    (UK_all['Exporters']=='World')  
   | (UK_all['Exporters']=='Austria') 
   | (UK_all['Exporters']=='Czech Republic')
   | (UK_all['Exporters']=='Italy')
   | (UK_all['Exporters']=='Turkey')
   | (UK_all['Exporters']=='China')        

    )]
    columnnames=UK_all.columns[UK_all.columns.str.contains("Unit")==True]
    UK_all.drop(columnnames, axis=1,inplace=True)
    UK_all=UK_all.transpose().reset_index()



In [93]:
# İlk Satırı Kolon isimleri yapar ve NaN olan satırları siler 
UK_all.columns = UK_all.iloc[0]
UK_all=UK_all.reindex(UK_all.index.drop(0))


#UK_all=UK_all.dropna(thresh=5)
UK_all.loc[:, 'Date'] = 0

columnindex=UK_all[UK_all['Exporters'].str.contains("Unnamed")==True].index
UK_all.drop(columnindex,inplace=True)

In [94]:
#if Product==841840 :
#    UK_all=UK_all.drop_duplicates(subset=['World', 'Germany'])
#else:
#    UK_all=UK_all.drop_duplicates('World')

In [95]:
UK_all[UK_all['Exporters']=='Unnamed: 21_x'].count()

0
Exporters             0
Turkey                0
Korea, Republic of    0
Poland                0
China                 0
Italy                 0
Date                  0
dtype: int64

In [96]:
UK_all.iloc[50:100,:]
#UK_

,Exporters,Turkey,"Korea, Republic of",Poland,China,Italy,Date
53,2014-M08-Imported unit value,3.87,5.78,4924,3.58,5979,0
54,2014-M09-Imported unit value,3.75,5.24,4847,3.32,5936,0
55,2014-M10-Imported unit value,3.67,5.4,4401,3.64,6253,0
56,2014-M11-Imported unit value,3.54,5.06,4545,3.58,5783,0
57,2014-M12-Imported unit value,3.4,4.57,4380,3.59,5665,0
58,2015-M01-Imported unit value,3.38,5.39,4331,3.62,5593,0
59,2015-M02-Imported unit value,3.29,5.22,4250,3.71,5421,0
60,2015-M03-Imported unit value,3.16,5.78,4300,3.45,5321,0
61,2015-M04-Imported unit value,3.14,5.76,4355,3.51,4727,0
62,2015-M05-Imported unit value_y,3.06,6.02,4361,3.51,4654,0


In [97]:
def finddate(expression):
        
    year=expression.split('-M')[0]
    year=year.split()[3]
    month=expression.split('-M')[1]
    month=month.split('_')[0]
    date=int(year+month)

    return date

In [98]:
for num in range(0,UK_all.shape[0]):
    UK_all.iloc[num,6]=finddate(UK_all.iloc[num,0])

IndexError: list index out of range

In [56]:
UK_all.iloc[num,0]

2.68

In [ ]:
UK_all.drop('Exporters',axis=1,inplace=True)
UK_all=UK_all.sort_values(by='Date', ascending=1)

UK_all=UK_all.drop_duplicates('Date')

In [ ]:
UK_all

In [53]:
if Product==841810 :
    UK_all=UK_all.reindex(columns=['Turkey','China','Poland','Korea, Republic of','Italy','Date'])

    UK_all=UK_all.rename(columns = {'Turkey':'TUR_GBR_841810'})
    UK_all['TUR_GBR_841810-1'] = UK_all['TUR_GBR_841810'].shift(1)
    UK_all['TUR_GBR_841810-2'] = UK_all['TUR_GBR_841810'].shift(2)
    UK_all['TUR_GBR_841810-3'] = UK_all['TUR_GBR_841810'].shift(3)
    UK_all['TUR_GBR_841810-6'] = UK_all['TUR_GBR_841810'].shift(6)
    UK_all['TUR_GBR_841810-12'] = UK_all['TUR_GBR_841810'].shift(12)

    UK_all=UK_all.rename(columns = {'China':'CHN_GBR_841810'})
    UK_all['CHN_GBR_841810-1'] = UK_all['CHN_GBR_841810'].shift(1)
    UK_all['CHN_GBR_841810-2'] = UK_all['CHN_GBR_841810'].shift(2)
    UK_all['CHN_GBR_841810-3'] = UK_all['CHN_GBR_841810'].shift(3)
    UK_all['CHN_GBR_841810-6'] = UK_all['CHN_GBR_841810'].shift(6)
    UK_all['CHN_GBR_841810-12'] = UK_all['CHN_GBR_841810'].shift(12)

    UK_all=UK_all.rename(columns = {'Poland':'POL_GBR_841810'})
    UK_all['POL_GBR_841810-1'] = UK_all['POL_GBR_841810'].shift(1)
    UK_all['POL_GBR_841810-2'] = UK_all['POL_GBR_841810'].shift(2)
    UK_all['POL_GBR_841810-3'] = UK_all['POL_GBR_841810'].shift(3)
    UK_all['POL_GBR_841810-6'] = UK_all['POL_GBR_841810'].shift(6)
    UK_all['POL_GBR_841810-12'] = UK_all['POL_GBR_841810'].shift(12)

    UK_all=UK_all.rename(columns = {'Korea, Republic of':'KOR_GBR_841810'})
    UK_all['KOR_GBR_841810-1'] = UK_all['KOR_GBR_841810'].shift(1)
    UK_all['KOR_GBR_841810-2'] = UK_all['KOR_GBR_841810'].shift(2)
    UK_all['KOR_GBR_841810-3'] = UK_all['KOR_GBR_841810'].shift(3)
    UK_all['KOR_GBR_841810-6'] = UK_all['KOR_GBR_841810'].shift(6)
    UK_all['KOR_GBR_841810-12'] = UK_all['KOR_GBR_841810'].shift(12)

    UK_all=UK_all.rename(columns = {'Italy':'ITA_GBR_841810'})
    UK_all['ITA_GBR_841810-1'] = UK_all['ITA_GBR_841810'].shift(1)
    UK_all['ITA_GBR_841810-2'] = UK_all['ITA_GBR_841810'].shift(2)
    UK_all['ITA_GBR_841810-3'] = UK_all['ITA_GBR_841810'].shift(3)
    UK_all['ITA_GBR_841810-6'] = UK_all['ITA_GBR_841810'].shift(6)
    UK_all['ITA_GBR_841810-12'] = UK_all['ITA_GBR_841810'].shift(12)

elif Product==841840:

    UK_all=UK_all.reindex(columns=['Turkey','China','Germany','Netherlands','Hungary','Date'])
    UK_all=UK_all.rename(columns = {'Turkey':'TUR_GBR_841840'})
    UK_all['TUR_GBR_841840-1'] = UK_all['TUR_GBR_841840'].shift(1)
    UK_all['TUR_GBR_841840-2'] = UK_all['TUR_GBR_841840'].shift(2)
    UK_all['TUR_GBR_841840-3'] = UK_all['TUR_GBR_841840'].shift(3)
    UK_all['TUR_GBR_841840-6'] = UK_all['TUR_GBR_841840'].shift(6)
    UK_all['TUR_GBR_841840-12'] = UK_all['TUR_GBR_841840'].shift(12)

    UK_all=UK_all.rename(columns = {'China':'CHN_GBR_841840'})
    UK_all['CHN_GBR_841840-1'] = UK_all['CHN_GBR_841840'].shift(1)
    UK_all['CHN_GBR_841840-2'] = UK_all['CHN_GBR_841840'].shift(2)
    UK_all['CHN_GBR_841840-3'] = UK_all['CHN_GBR_841840'].shift(3)
    UK_all['CHN_GBR_841840-6'] = UK_all['CHN_GBR_841840'].shift(6)
    UK_all['CHN_GBR_841840-12'] = UK_all['CHN_GBR_841840'].shift(12)

    
    UK_all=UK_all.rename(columns = {'Germany':'DEU_GBR_841840'})
    UK_all['DEU_GBR_841840-1'] = UK_all['DEU_GBR_841840'].shift(1)
    UK_all['DEU_GBR_841840-2'] = UK_all['DEU_GBR_841840'].shift(2)
    UK_all['DEU_GBR_841840-3'] = UK_all['DEU_GBR_841840'].shift(3)
    UK_all['DEU_GBR_841840-6'] = UK_all['DEU_GBR_841840'].shift(6)
    UK_all['DEU_GBR_841840-12'] = UK_all['DEU_GBR_841840'].shift(12)

    UK_all=UK_all.rename(columns = {'Netherlands':'NLD_GBR_841840'})
    UK_all['NLD_GBR_841840-1'] = UK_all['NLD_GBR_841840'].shift(1)
    UK_all['NLD_GBR_841840-2'] = UK_all['NLD_GBR_841840'].shift(2)
    UK_all['NLD_GBR_841840-3'] = UK_all['NLD_GBR_841840'].shift(3)
    UK_all['NLD_GBR_841840-6'] = UK_all['NLD_GBR_841840'].shift(6)
    UK_all['NLD_GBR_841840-12'] = UK_all['NLD_GBR_841840'].shift(12)

    UK_all=UK_all.rename(columns = {'Hungary':'HUN_GBR_841840'})
    UK_all['HUN_GBR_841840-1'] = UK_all['HUN_GBR_841840'].shift(1)
    UK_all['HUN_GBR_841840-2'] = UK_all['HUN_GBR_841840'].shift(2)
    UK_all['HUN_GBR_841840-3'] = UK_all['HUN_GBR_841840'].shift(3)
    UK_all['HUN_GBR_841840-6'] = UK_all['HUN_GBR_841840'].shift(6)
    UK_all['HUN_GBR_841840-12'] = UK_all['HUN_GBR_841840'].shift(12)
    
elif Product==841850:
    UK_all=UK_all.reindex(columns=['Turkey','China','Austria','Czech Republic','Italy','Date'])
    UK_all=UK_all.rename(columns = {'Turkey':'TUR_GBR_841850'})
    UK_all['TUR_GBR_841850-1'] = UK_all['TUR_GBR_841850'].shift(1)
    UK_all['TUR_GBR_841850-2'] = UK_all['TUR_GBR_841850'].shift(2)
    UK_all['TUR_GBR_841850-3'] = UK_all['TUR_GBR_841850'].shift(3)
    UK_all['TUR_GBR_841850-6'] = UK_all['TUR_GBR_841850'].shift(6)
    UK_all['TUR_GBR_841850-12'] = UK_all['TUR_GBR_841850'].shift(12)

    UK_all=UK_all.rename(columns = {'China':'CHN_GBR_841850'})
    UK_all['CHN_GBR_841850-1'] = UK_all['CHN_GBR_841850'].shift(1)
    UK_all['CHN_GBR_841850-2'] = UK_all['CHN_GBR_841850'].shift(2)
    UK_all['CHN_GBR_841850-3'] = UK_all['CHN_GBR_841850'].shift(3)
    UK_all['CHN_GBR_841850-6'] = UK_all['CHN_GBR_841850'].shift(6)
    UK_all['CHN_GBR_841850-12'] = UK_all['CHN_GBR_841850'].shift(12)

    
    UK_all=UK_all.rename(columns = {'Austria':'AUT_GBR_841850'})
    UK_all['AUT_GBR_841850-1'] = UK_all['AUT_GBR_841850'].shift(1)
    UK_all['AUT_GBR_841850-2'] = UK_all['AUT_GBR_841850'].shift(2)
    UK_all['AUT_GBR_841850-3'] = UK_all['AUT_GBR_841850'].shift(3)
    UK_all['AUT_GBR_841850-6'] = UK_all['AUT_GBR_841850'].shift(6)
    UK_all['AUT_GBR_841850-12'] = UK_all['AUT_GBR_841850'].shift(12)

    UK_all=UK_all.rename(columns = {'Czech Republic':'CHZ_GBR_841850'})
    UK_all['CHZ_GBR_841850-1'] = UK_all['CHZ_GBR_841850'].shift(1)
    UK_all['CHZ_GBR_841850-2'] = UK_all['CHZ_GBR_841850'].shift(2)
    UK_all['CHZ_GBR_841850-3'] = UK_all['CHZ_GBR_841850'].shift(3)
    UK_all['CHZ_GBR_841850-6'] = UK_all['CHZ_GBR_841850'].shift(6)
    UK_all['CHZ_GBR_841850-12'] = UK_all['CHZ_GBR_841850'].shift(12)

    UK_all=UK_all.rename(columns = {'Italy':'ITA_GBR_841850'})
    UK_all['ITA_GBR_841850-1'] = UK_all['ITA_GBR_841850'].shift(1)
    UK_all['ITA_GBR_841850-2'] = UK_all['ITA_GBR_841850'].shift(2)
    UK_all['ITA_GBR_841850-3'] = UK_all['ITA_GBR_841850'].shift(3)
    UK_all['ITA_GBR_841850-6'] = UK_all['ITA_GBR_841850'].shift(6)
    UK_all['ITA_GBR_841850-12'] = UK_all['ITA_GBR_841850'].shift(12)


In [54]:
UK_all=UK_all.drop_duplicates('Date')
#UK_all=UK_all.dropna()
UK_all.to_excel('{}_GBR_all_unit_value.xlsx'.format(Product),index = False)